In [1]:
from cassandra.cluster import Cluster

cluster = Cluster()

In [2]:
session = cluster.connect()

In [3]:
session.execute("SELECT release_version FROM system.local").one()

Row(release_version='3.11.11')

In [4]:
session.execute("select * from store.shopping_cart").all()

[Row(userid='1234', item_count=5, last_update_timestamp=datetime.datetime(2022, 2, 21, 8, 13, 46, 714000)),
 Row(userid='9876', item_count=2, last_update_timestamp=datetime.datetime(2022, 2, 21, 8, 13, 46, 705000))]

In [5]:
session.set_keyspace('store')

session.execute("select * from shopping_cart").all()

[Row(userid='1234', item_count=5, last_update_timestamp=datetime.datetime(2022, 2, 21, 8, 13, 46, 714000)),
 Row(userid='9876', item_count=2, last_update_timestamp=datetime.datetime(2022, 2, 21, 8, 13, 46, 705000))]

In [6]:
session.execute("select * from user").all()

[Row(last_name='Rodriguez', first_name='Mary', address=('7712 E. Broadway', 'Tucson', 'AZ', 85715), addresses=OrderedMapSerializedKey([('home', address(street='7112 E. Broadway', city='Tucson', state='AZ', zip_code=85715))]), emails=SortedSet(['mary.Rodriguez.AZ@gmail.com', 'mary@example.com']), id=UUID('bcc50922-fd55-4811-a123-f2af786cc73a'), login_sessions=OrderedMapSerializedKey([(UUID('3246b850-92ee-11ec-aac8-11ca1d03e536'), 13), (UUID('3246b851-92ee-11ec-aac8-11ca1d03e536'), 18)]), middle_initial=None, phone_numbers=['480-111-1111'], title=None),
 Row(last_name='Nguyen', first_name='Bill', address=None, addresses=None, emails=None, id=None, login_sessions=None, middle_initial='R', phone_numbers=None, title='Mr.'),
 Row(last_name='Nguyen', first_name='Wanda', address=None, addresses=None, emails=None, id=None, login_sessions=None, middle_initial=None, phone_numbers=None, title='Mrs.')]

In [7]:
%env JAVA_HOME=/usr/local/lib/java/Contents/Home

env: JAVA_HOME=/usr/local/lib/java/Contents/Home


In [8]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.setAppName('local')
conf.set('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions')
conf.set('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.1.0')

sc = SparkContext(conf=conf)
sc

:: loading settings :: url = jar:file:/Users/kimmy.liu/Workspace/playz3/.venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/kimmy.liu/.ivy2/cache
The jars for the packages stored in: /Users/kimmy.liu/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-238bd38b-12bf-4959-b468-15b865255d45;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.1.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.1.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.12.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in c

<SparkContext master=local[*] appName=local>

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession(sparkContext=sc)

spark.conf.set("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog")

spark

In [10]:
spark.sql("CREATE DATABASE IF NOT EXISTS mycatalog.testks WITH DBPROPERTIES (class='SimpleStrategy',replication_factor='1')")

DataFrame[]

In [11]:
spark.sql('show namespaces from mycatalog').show()

+------------------+
|         namespace|
+------------------+
|       reservation|
|            testks|
|           cadence|
|             hotel|
|             store|
|cadence_visibility|
+------------------+



In [12]:
df = spark.read.table("mycatalog.store.shopping_cart")

In [13]:
df.show()

+------+----------+---------------------+
|userid|item_count|last_update_timestamp|
+------+----------+---------------------+
|  9876|         2| 2022-02-21 16:13:...|
|  1234|         5| 2022-02-21 16:13:...|
+------+----------+---------------------+



In [14]:
df.select("userid").show()

+------+
|userid|
+------+
|  9876|
|  1234|
+------+



In [15]:
df.summary().show()

+-------+-----------------+------------------+
|summary|           userid|        item_count|
+-------+-----------------+------------------+
|  count|                2|                 2|
|   mean|           5555.0|               3.5|
| stddev|6110.816803014143|2.1213203435596424|
|    min|             1234|                 2|
|    25%|           1234.0|                 2|
|    50%|           1234.0|                 2|
|    75%|           9876.0|                 5|
|    max|             9876|                 5|
+-------+-----------------+------------------+



In [16]:
df.dtypes

[('userid', 'string'),
 ('item_count', 'int'),
 ('last_update_timestamp', 'timestamp')]

In [17]:
userDf = spark.read.format("org.apache.spark.sql.cassandra").option("keyspace", "store") \
    .option("table", "user")\
    .load()

In [18]:
userDf.printSchema()

root
 |-- last_name: string (nullable = false)
 |-- first_name: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- 0: string (nullable = true)
 |    |-- 1: string (nullable = true)
 |    |-- 2: string (nullable = true)
 |    |-- 3: integer (nullable = true)
 |-- addresses: map (nullable = true)
 |    |-- key: string
 |    |-- value: struct (valueContainsNull = true)
 |    |    |-- street: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- zip_code: integer (nullable = true)
 |-- emails: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- login_sessions: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)
 |-- middle_initial: string (nullable = true)
 |-- phone_numbers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)



In [19]:
userDf.show()

+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----+
|last_name|first_name|             address|           addresses|              emails|                  id|      login_sessions|middle_initial| phone_numbers|title|
+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----+
|Rodriguez|      Mary|{7712 E. Broadway...|{home -> {7112 E....|[mary.Rodriguez.A...|bcc50922-fd55-481...|{3246b850-92ee-11...|          null|[480-111-1111]| null|
|   Nguyen|      Bill|                null|                  {}|                  []|                null|                  {}|             R|            []|  Mr.|
|   Nguyen|     Wanda|                null|                  {}|                  []|                null|                  {}|          null|            []| Mrs.|
+---------+-----

In [20]:
from pyspark.sql.functions import isnull

userDf.filter(~isnull(userDf['middle_initial'])).show()

+---------+----------+-------+---------+------+----+--------------+--------------+-------------+-----+
|last_name|first_name|address|addresses|emails|  id|login_sessions|middle_initial|phone_numbers|title|
+---------+----------+-------+---------+------+----+--------------+--------------+-------------+-----+
|   Nguyen|      Bill|   null|       {}|    []|null|            {}|             R|           []|  Mr.|
+---------+----------+-------+---------+------+----+--------------+--------------+-------------+-----+



In [21]:
familyDf = userDf.groupby("last_name").agg({"*": "count"}).withColumnRenamed("COUNT(1)", "members")
familyDf.show()

+---------+-------+
|last_name|members|
+---------+-------+
|   Nguyen|      2|
|Rodriguez|      1|
+---------+-------+



In [22]:
session.execute("CREATE TABLE IF NOT EXISTS store.family (last_name text primary key, members int)").all()

familyDf.write.format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .option("keyspace", "store") \
    .option("table", "family").save()

In [23]:
session.execute("select * from store.family").all()

[Row(last_name='Rodriguez', members=1), Row(last_name='Nguyen', members=2)]